In [14]:
# Install required packages
!pip install requests
!rm -f gists.db  # Remove existing database file

import requests
import sqlite3
import logging
import os
from datetime import datetime
from typing import List, Dict, Optional, Any

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class Gist:
    """Represents a GitHub Gist with all relevant fields"""
    def __init__(self, github_id: str, html_url: str, git_pull_url: str,
                 git_push_url: str, commits_url: str, forks_url: str,
                 public: bool, created_at: str, updated_at: str,
                 comments: int, comments_url: str):
        self.github_id = github_id
        self.html_url = html_url
        self.git_pull_url = git_pull_url
        self.git_push_url = git_push_url
        self.commits_url = commits_url
        self.forks_url = forks_url
        self.public = public
        self.created_at = created_at
        self.updated_at = updated_at
        self.comments = comments
        self.comments_url = comments_url

    def __repr__(self):
        return f"Gist(id={self.github_id}, created={self.created_at[:10]})"

def initialize_database(db_path: str = 'gists.db') -> sqlite3.Connection:
    """Initialize fresh SQLite database with clean schema"""
    # Remove old database if exists
    if os.path.exists(db_path):
        os.remove(db_path)

    conn = sqlite3.connect(db_path)

    # Create new table with explicit columns
    conn.execute("""
    CREATE TABLE gists (
        github_id TEXT PRIMARY KEY,
        html_url TEXT NOT NULL,
        git_pull_url TEXT NOT NULL,
        git_push_url TEXT NOT NULL,
        commits_url TEXT NOT NULL,
        forks_url TEXT NOT NULL,
        public BOOLEAN NOT NULL,
        created_at TEXT NOT NULL,
        updated_at TEXT NOT NULL,
        comments INTEGER NOT NULL,
        comments_url TEXT NOT NULL
    );
    """)

    # Create indexes
    conn.execute("CREATE INDEX idx_created ON gists(created_at)")
    conn.execute("CREATE INDEX idx_updated ON gists(updated_at)")
    conn.commit()
    return conn

def fetch_all_gists(username: str) -> List[Dict]:
    """Fetch all gists for a GitHub user with pagination"""
    url = f"https://api.github.com/users/{username}/gists"
    all_gists = []

    try:
        while url:
            logger.info(f"Fetching gists from: {url}")
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            # Check rate limits
            if int(response.headers.get('X-RateLimit-Remaining', 1)) < 5:
                reset_time = datetime.fromtimestamp(int(response.headers['X-RateLimit-Reset']))
                sleep_time = max(0, (reset_time - datetime.now()).total_seconds()) + 5
                logger.warning(f"Approaching rate limit. Sleeping for {sleep_time} seconds")
                time.sleep(sleep_time)

            all_gists.extend(response.json())
            url = response.links.get('next', {}).get('url', None)
            time.sleep(0.5)  # Respect GitHub API limits

        logger.info(f"Successfully fetched {len(all_gists)} gists")
        return all_gists

    except Exception as e:
        logger.error(f"Failed to fetch gists: {str(e)}")
        raise

def import_gists_to_database(db: sqlite3.Connection, username: str, commit: bool = True) -> int:
    """Import user's gists into the database"""
    gists = fetch_all_gists(username)
    if not gists:
        logger.warning(f"No gists found for user {username}")
        return 0

    try:
        cursor = db.cursor()
        # Explicit column names in INSERT statement
        cursor.executemany("""
            INSERT INTO gists (
                github_id, html_url, git_pull_url, git_push_url,
                commits_url, forks_url, public, created_at,
                updated_at, comments, comments_url
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, [
            (
                gist['id'],
                gist['html_url'],
                gist['git_pull_url'],
                gist['git_push_url'],
                gist['commits_url'],
                gist['forks_url'],
                int(gist['public']),
                gist['created_at'],
                gist['updated_at'],
                gist['comments'],
                gist['comments_url']
            ) for gist in gists
        ])

        if commit:
            db.commit()

        logger.info(f"Imported {len(gists)} gists for user {username}")
        return len(gists)

    except Exception as e:
        logger.error(f"Database error: {str(e)}")
        db.rollback()
        raise

def search_gists(db: sqlite3.Connection,
                github_id: Optional[str] = None,
                created_at: Optional[str] = None,
                **filters) -> List[Gist]:
    """Search gists with flexible filtering"""
    query = "SELECT * FROM gists WHERE 1=1"
    params = {}

    # Basic filters
    if github_id:
        query += " AND github_id = :github_id"
        params['github_id'] = github_id

    if created_at:
        query += " AND datetime(created_at) = datetime(:created_at)"
        params['created_at'] = created_at

    # Date range filters
    date_fields = {
        'created_at': ['gt', 'gte', 'lt', 'lte'],
        'updated_at': ['gt', 'gte', 'lt', 'lte']
    }

    for field, operators in date_fields.items():
        for op in operators:
            key = f"{field}__{op}"
            if key in filters:
                sql_op = {'gt': '>', 'gte': '>=', 'lt': '<', 'lte': '<='}[op]
                query += f" AND datetime({field}) {sql_op} datetime(:{key})"
                params[key] = filters[key]

    try:
        cursor = db.execute(query, params)
        columns = [col[0] for col in cursor.description]
        return [Gist(**dict(zip(columns, row))) for row in cursor.fetchall()]

    except Exception as e:
        logger.error(f"Search failed: {str(e)}")
        raise

# Main execution
if __name__ == "__main__":
    try:
        # Initialize fresh database
        db = initialize_database()

        # Import gists (example with Python creator's account)
        import_gists_to_database(db, "gvanrossum")

        # Search examples
        print("\n=== Recent gists (after Jan 1, 2023) ===")
        recent_gists = search_gists(db, created_at__gt="2023-01-01T00:00:00Z")
        for gist in recent_gists[:5]:  # Show first 5 results
            print(f"- {gist.html_url} (Created: {gist.created_at[:10]})")

        print("\n=== All gists count ===")
        all_gists = search_gists(db)
        print(f"Total gists in database: {len(all_gists)}")

    except Exception as e:
        logger.error(f"Application error: {str(e)}")
    finally:
        db.close()
        logger.info("Database connection closed")


=== Recent gists (after Jan 1, 2023) ===

=== All gists count ===
Total gists in database: 12
